<div align="center" style="border:solid 1px gray;">
    <a href="https://openalex.org/">
        <img src="../../resources/img/OpenAlex-banner.png" alt="OpenAlex banner" width="300">
    </a>
</div>

!!! Attention: In order for the visualizations at the end of this tutorial to render properly, you should be viewing this using nbviewer, by clicking on this link, if you haven't already: <https://nbviewer.org/github/ourresearch/openalex-api-tutorials/blob/main/notebooks/institutions/uw-collaborators.ipynb>

# How are my institution's researchers collaborating with people around the globe?

In this tutorial, we will use OpenAlex's API to get all of the publications that are collaborations between a given institution and other institutions around the world.

We will use the University of Washington (UW) as our example institution. This is a **real-world example:** The UW Office of Global Affairs has built an [interactive publications dashboard](https://www.washington.edu/global/publications/) which lets people dig deep into these collaborations using maps, graphs, and other visualizations. This lets them show off the scholarly collaborations between UW's faculty and researchers around the world.

The data powering this dashboard is based on the Microsoft Academic Graph, which is no longer being updated. So we will use OpenAlex's API to get freshly updated data for UW.


First, let's look at how the University of Washington is represented in OpenAlex.

[Institutions](https://docs.openalex.org/api-entities/institutions) in OpenAlex are closely linked with the [ROR registry](https://ror.org/) of research organizations. By searching the ROR website, we can find the [ROR ID for UW](https://ror.org/00cvxb145).

In [1]:
uw_id = "https://ror.org/00cvxb145"

## Get the data for UW publications

To get the data for the UW dashboard, we need to collect all of the works from OpenAlex which have at least one author from UW and at least one author outside UW. We'll do this in two steps: first, we'll get all of the works with UW authors, then we'll filter to keep only the papers with at least one other affiliation.

Make sure to put your email address in the `email` variable. This is how you use the [polite pool](https://docs.openalex.org/how-to-use-the-api/rate-limits-and-authentication#the-polite-pool), which will get you faster and more consistent response times when using the API.

In [2]:
### ADD YOUR EMAIL to use the polite pool
email = "jportenoy@ourresearch.org"
# email = ""

# specify endpoint
endpoint = 'works'

# build the 'filter' parameter
# We'll limit it to the last 20 years
filters = ",".join((
    f'institutions.ror:{uw_id}',
    'from_publication_date:2003-01-01',
))

# put the URL together
filtered_works_url = f'https://api.openalex.org/{endpoint}?filter={filters}'
if email:
    filtered_works_url += f"&mailto={email}"
print(f'complete URL with filters:\n{filtered_works_url}')

complete URL with filters:
https://api.openalex.org/works?filter=institutions.ror:https://ror.org/00cvxb145,from_publication_date:2003-01-01&mailto=jportenoy@ourresearch.org


We've built the URL. We are requesting a list of Works from the API, using two [filters](https://docs.openalex.org/api-entities/works/filter-works) to specify which works we want. If you like, [you can use your browser](https://docs.openalex.org/how-to-use-the-api/rate-limits-and-authentication#calling-the-api-in-your-browser) (such as Firefox or Chrome) to go to the URL we've built and view the data. (Make sure you have a browser extension which formats JSON in a nice way. Otherwise the data will look like a big mess on your screen.)

We're more interested in getting the data into Python. So let's get the results from the API, using the `requests` library.

In [3]:
import requests

In [4]:
r = requests.get(filtered_works_url)
results_page = r.json()
print(f"retrieved {len(results_page['results'])} works")

retrieved 25 works


We've retrieved 25 works, but of course that isn't the total number of works by UW authors. We only got the first page of results. We can see how many works there actually are by looking at the `results_page['meta']['count']` value:

In [5]:
print(results_page['meta']['count'])

256519


In [6]:
# there are 25 results per page
results_page['meta']['count'] / 25

10260.76

### Get the full publications data

There are about 250,000 works. To get all of them, we will need to [get multiple pages of results](https://docs.openalex.org/how-to-use-the-api/get-lists-of-entities/paging). Since we want more than 10,000 works, we need to use the cursor paging technique. At 25 results per page, that means we will need to make nearly 10,000 API calls to get all of the data we want.

This may seem like a lot, but don't panic! This is well under the [free allowance of 100,000 API calls per day.](https://docs.openalex.org/how-to-use-the-api/rate-limits-and-authentication) It will take about an hour in total, so let it run overnight or while you take a break.


In [7]:
cursor = '*'

select = ",".join((
    'id',
    'ids',
    'title',
    'display_name',
    'publication_year',
    'publication_date',
    'primary_location',
    'open_access',
    'authorships',
    'cited_by_count',
    'is_retracted',
    'is_paratext',
    'updated_date',
    'created_date',
))

# loop through pages
works = []
loop_index = 0
while cursor:
    
    # set cursor value and request page from OpenAlex
    url = f'{filtered_works_url}&select={select}&cursor={cursor}'
    page_with_results = requests.get(url).json()
    
    results = page_with_results['results']
    works.extend(results)

    # update cursor to meta.next_cursor
    cursor = page_with_results['meta']['next_cursor']
    loop_index += 1
    if loop_index in [5, 10, 20, 50, 100] or loop_index % 500 == 0:
        print(f'{loop_index} api requests made so far')
print(f'done. made {loop_index} api requests. collected {len(works)} works')

5 api requests made so far
10 api requests made so far
20 api requests made so far
50 api requests made so far
100 api requests made so far
500 api requests made so far
1000 api requests made so far
1500 api requests made so far
2000 api requests made so far
2500 api requests made so far
3000 api requests made so far
3500 api requests made so far
4000 api requests made so far
4500 api requests made so far
5000 api requests made so far
5500 api requests made so far
6000 api requests made so far
6500 api requests made so far
7000 api requests made so far
7500 api requests made so far
8000 api requests made so far
8500 api requests made so far
9000 api requests made so far
9500 api requests made so far
10000 api requests made so far
done. made 10262 api requests. collected 256521 works


### Save the raw data for later (optional)

Now might be a good time to save the data we just retrieved, in case this notebook restarts and we want to come back to it without having to get all of the works from the API again. The following cell allows us to do this by saving the data using the [`.pickle`](https://docs.python.org/3/library/pickle.html) data format. (If you are loading the data in this way, then you don't need to run the above cell.)

In [8]:
import pickle

# uncomment these lines and run to save the results so we won't have to fetch them
# again next time we run the notebook
# import os
# if not os.path.isdir('../../data'):
#     os.mkdir('../../data')
# with open('../../data/uw_works_since_2003.pickle', 'wb') as outf:
#     pickle.dump(works, outf, protocol=pickle.HIGHEST_PROTOCOL)

# OR uncomment these lines and run to load the saved results
# with open('../../data/uw_works_since_2003.pickle', 'rb') as f:
#     works = pickle.load(f)

### Use Pandas to organize the data

Each work object has a lot of information, not all of which we will use. Let's get the data into a [Pandas dataframe](https://pandas.pydata.org/), limiting the number of fields to just the ones we might be interested in.

In [9]:
import pandas as pd
data = []
for work in works:
    for authorship in work['authorships']:
        if authorship:
            author = authorship['author']
            author_id = author['id'] if author else None
            author_name = author['display_name'] if author else None
            author_position = authorship['author_position']
            for institution in authorship['institutions']:
                if institution:
                    institution_id = institution['id']
                    institution_name = institution['display_name']
                    institution_country_code = institution['country_code']
                    data.append({
                        'work_id': work['id'],
                        'work_title': work['title'],
                        'work_display_name': work['display_name'],
                        'work_publication_year': work['publication_year'],
                        'work_publication_date': work['publication_date'],
                        'author_id': author_id,
                        'author_name': author_name,
                        'author_position': author_position,
                        'institution_id': institution_id,
                        'institution_name': institution_name,
                        'institution_country_code': institution_country_code,
                    })
df = pd.DataFrame(data)

In [10]:
df

,work_id,work_title,work_display_name,work_publication_year,work_publication_date,author_id,author_name,author_position,institution_id,institution_name,institution_country_code
0,https://openalex.org/W2142225512,Three Approaches to Qualitative Content Analysis,Three Approaches to Qualitative Content Analysis,2005,2005-11-01,https://openalex.org/A2103438708,Hsiu-Fang Hsieh,first,https://openalex.org/I64045040,Fooyin University,TW
1,https://openalex.org/W2142225512,Three Approaches to Qualitative Content Analysis,Three Approaches to Qualitative Content Analysis,2005,2005-11-01,https://openalex.org/A4355292137,Sarah E. Shannon,last,https://openalex.org/I201448701,University of Washington,US
2,https://openalex.org/W2963037989,"You Only Look Once: Unified, Real-Time Object ...","You Only Look Once: Unified, Real-Time Object ...",2016,2016-06-27,https://openalex.org/A4340105427,Joseph Redmon,first,https://openalex.org/I201448701,University of Washington,US
3,https://openalex.org/W2963037989,"You Only Look Once: Unified, Real-Time Object ...","You Only Look Once: Unified, Real-Time Object ...",2016,2016-06-27,https://openalex.org/A2310010008,Santosh K. Divvala,middle,https://openalex.org/I4210140341,Allen Institute,US
4,https://openalex.org/W2963037989,"You Only Look Once: Unified, Real-Time Object ...","You Only Look Once: Unified, Real-Time Object ...",2016,2016-06-27,https://openalex.org/A2310010008,Santosh K. Divvala,middle,https://openalex.org/I201448701,University of Washington,US
...,...,...,...,...,...,...,...,...,...,...,...
1828922,https://openalex.org/W99980466,LASIK: Early Postoperative Complications,LASIK: Early Postoperative Complications,2008,2008-12-01,https://openalex.org/A4351033097,Oscar Gris,middle,https://openalex.org/I4210131277,Instituto de Microcirugía Ocular,ES
1828923,https://openalex.org/W99980466,LASIK: Early Postoperative Complications,LASIK: Early Postoperative Complications,2008,2008-12-01,https://openalex.org/A4338409353,Javier Gaytan,middle,https://openalex.org/I4210131277,Instituto de Microcirugía Ocular,ES
1828924,https://openalex.org/W99980466,LASIK: Early Postoperative Complications,LASIK: Early Postoperative Complications,2008,2008-12-01,https://openalex.org/A1993526635,Felicidad Manero,last,https://openalex.org/I4210131277,Instituto de Microcirugía Ocular,ES
1828925,https://openalex.org/W999903335,The Ecological Impact of the Korean Saemaul (N...,The Ecological Impact of the Korean Saemaul (N...,2015,2015-01-01,https://openalex.org/A4357860343,Chung Yong Kim,first,https://openalex.org/I58610484,Seattle University,US


### Filter for outside collaborations

We are only interested in publications which have at least one authorship outside of UW, so let's label each row with this information.

In [11]:
def outside_uw_collab(institution_ids):
    # Function that takes institution IDs (grouped by works)
    # and returns True if the work has at least one non-UW affiliation
    if all(institution_ids == 'https://openalex.org/I201448701'):
        return False
    else:
        return True
# transform() will return a series the same length as the dataframe, 
# which we can store as a column in the dataframe
df['is_outside_uw_collab'] = df.groupby('work_id')['institution_id'].transform(outside_uw_collab)

Now let's create a new dataframe with only the True rows (works that are collaborations).

In [12]:
df_collab = df[df['is_outside_uw_collab']].drop(columns='is_outside_uw_collab')
print(f"dataframe has {len(df_collab):,} rows, with {df_collab['work_id'].nunique():,} unique publications")

dataframe has 1,656,698 rows, with 161,807 unique publications


As a side note, we may be curious about how many of these collaborations are with institutions *outside the US,* rather than just outside UW. We can use a similar technique to answer this question.

In [13]:
def international_collab(institution_country_codes):
    # Function that takes institution country codes (grouped by works)
    # and returns True if the work has at least one non-US affiliated institution
    if all(institution_country_codes == 'US'):
        return False
    else:
        return True
df_collab['is_international_collab'] = df_collab.groupby('work_id')['institution_country_code'].transform(international_collab)
num_international_collab = df_collab[df_collab['is_international_collab']]['work_id'].nunique()
print(f"There are {num_international_collab:,} publications with international collaboration.")

There are 97,254 publications with international collaboration.


## Get the data for Institutions

In [14]:
print(f"There are {df_collab['institution_id'].nunique():,} institutions that collaborate with UW, in {df_collab['institution_country_code'].nunique()} countries.")

There are 22,983 institutions that collaborate with UW, in 200 countries.


### Query the API for institutions data

We've found about 20K institutions that collaborate with UW. We already know the names and the countries of these institutions; these were included in the results from the `works` endpoint of the API. There is some more information about the institutions that would be helpful to supply to the dashboard, especially the geolocation data that can be used to show the collaborations on maps. This information can be found in the [Institutions](https://docs.openalex.org/api-entities/institutions) entities.

To get the institutions, we will query the `/institutions` endpoint of the API. We will use the technique to get multiple entities per API request as suggested [here](https://docs.openalex.org/how-to-use-the-api/get-lists-of-entities/filter-entity-lists#addition-or). This technique involves getting 50 results at once by requesting them in a filter, separated by a pipe ('`|`').

This will get all of the institutions in about 400 API calls, which should only take a few minutes.

In [15]:
institution_ids = df['institution_id'].dropna().unique()

endpoint = "institutions"
size = 50
loop_index = 0
institutions = []
for list_index in range(0, len(institution_ids), size):
    subset = institution_ids[list_index:list_index+size]
    pipe_separated_ids = "|".join(subset)
    r = requests.get(f"https://api.openalex.org/institutions?filter=openalex:{pipe_separated_ids}&per-page={size}")
    results = r.json()['results']
    institutions.extend(results)
    loop_index += 1
print(f"collected {len(institutions)} institutions using {loop_index} api calls")

collected 21958 institutions using 460 api calls


Again, let's put this data into a Pandas dataframe.

In [16]:
data = []
for institution in institutions:
    data.append({
        'id': institution['id'],
        'ror': institution['ror'],
        'display_name': institution['display_name'],
        'country_code': institution['country_code'],
        'type': institution['type'],
        'latitude': institution['geo']['latitude'],
        'longitude': institution['geo']['longitude'],
        'city': institution['geo']['city'],
        'region': institution['geo']['region'],
        'country': institution['geo']['country'],
        'image_url': institution['image_url'],
        'image_thumbnail_url': institution['image_thumbnail_url'],
    })

df_institutions = pd.DataFrame(data)

## Save the data

Now it's time to save the data, so it can be handed off to the team at UW and be imported into their dashboard. We'll save two different files:
- The publications data will have one row for each publication-author-affiliation triplet. Each publication will have multiple rows, because there are multiple authors per paper. An author in a given publication could have more than one row, if they have more than one affiliation.
- The institutions data will have one row per institution, connected to the publications data by the `institution_id` column.

In [17]:
import os
if not os.path.isdir('../../data'):
    os.mkdir('../../data')
    
# Save the publications data
# Each row represents a publication-author-affiliation
#
# Compressing the data as a GZIP file will greatly reduce the file size.
# Pandas knows to use GZIP compression if the file name ends in `.gz`
outpath = '../../data/uw_collabs.csv.gz'
df_collab.to_csv(outpath, index=False)

# Save the institutions data
outpath = '../../data/uw_collabs_institutions.csv.gz'
df_institutions.to_csv(outpath, index=False)

## Visualize the data

While the UW's own dashboard does a great job of visualizing the data in sophisticated ways, it's also fun to whip up a few quick visualizations of our own. We can start with simply showing the number of international collaborations among UW publications over time.

### Collaborations over time

We'll use the python visualization library [Plotly](https://plotly.com/python/)—specifically, the [`plotly.express`](https://plotly.com/python/plotly-express/) module. It is very easy to get up and running visualizing our data, and as a bonus, the plots are interactive. If the interactive charts are not rendering properly, please use the nbviewer renderer (see the beginning of this notebook).

In [ ]:
import plotly.express as px

In [48]:
# Count the international collaborations by year
international_collabs_by_year = df_collab.groupby('work_publication_year')['is_international_collab'].sum().reset_index(name='count')

# Exclude the year 2023
international_collabs_by_year = international_collabs_by_year[international_collabs_by_year['work_publication_year']<2023]

fig = px.line(international_collabs_by_year, x='work_publication_year', y='count')
fig.show()

Next, let's plot the same data, but this time by country. We'll plot the top ten countries.

We'll use the [country_converter](https://github.com/IndEcol/country_converter) python library to get more readable names for the countries.

In [49]:
import country_converter as coco



In [50]:
# Get the counts by year by country
country_by_year = df_collab.groupby(['work_publication_year', 'institution_country_code']).size().reset_index(name='count')

# Get the names of the countries from the country codes
cc = coco.CountryConverter()
country_by_year['country'] = cc.pandas_convert(country_by_year['institution_country_code'], to='name_short')

# Filter out collaborations with other US institutions
country_by_year = country_by_year[country_by_year['institution_country_code']!='US']

# Exclude the year 2023
country_by_year = country_by_year[country_by_year['work_publication_year']<2023]

# Get the top ten countries
top_countries = country_by_year.groupby('institution_country_code')['count'].sum().sort_values(ascending=False).head(10).index
country_by_year_subset = country_by_year[country_by_year['institution_country_code'].isin(top_countries)]

# Plot the data
fig = px.line(country_by_year_subset, x='work_publication_year', y='count', color='country')
fig.show()

### Chloropleth map

Finally, let's make a [chloropleth map](https://en.wikipedia.org/wiki/Choropleth_map), which uses color to show the number of collaborations in countries around the world.

In [18]:
country_counts = df_collab['institution_country_code'].value_counts()
country_counts = country_counts[country_counts.index!='US']

In [19]:
country_counts = country_counts.rename_axis('country_code').reset_index(name='num_collaborations')

In [51]:
# Get the names of the countries from the country codes
cc = coco.CountryConverter()
country_counts['country'] = cc.pandas_convert(country_counts['country_code'], to='name_short')
# The Plotly chloropleth method needs the ISO3 (3-letter) country code
country_counts['country_code_ISO3'] = cc.pandas_convert(country_counts['country_code'], to='ISO3')

# Make the map
fig = px.choropleth(country_counts, locations='country_code_ISO3', locationmode='ISO-3', color='num_collaborations', hover_name='country')
fig.show()